# K-Means Clustering Notebook

## About

With this project I am to use the K-Means clustering algorithm on the mall customer dataset. The dataset features information about customers such as their age, annual income and spending core. The goal of this notebook will be to understand more about the customers so that they can be offered products in a more tailored way. 

## Setup

### Import libraries and dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../datasets/mall_customer_segmentation_data/Mall_Customers.csv')

In [3]:
data.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


### Graph Data